# Process images to extract spectral features

April 13, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob

In [2]:
%matplotlib widget

In [4]:
pwd

'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/6_spectral_analysis'

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

[NbConvertApp] WARNING | pattern 'modules_image_analysis.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook outp

In [80]:
### Get imput image

##### From file #####
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
# f1='20200513_121910_peters_dataset/dump_outs/model0-validation-epoch38-step11972-gen_img_instance1_activation-output0.npy'
f1='20200513_121910_peters_dataset/dump_outs/model0-validation-epoch38-step11972-inp_img-output0.npy'
fname=main_dir+f1
a1=np.load(fname)
print(a1.shape)
a1=a1.reshape(128,128,128)
img=a1[0]
print(img.shape)

(128, 16384)
(128, 128)


To do : 
- Read an image
- Compute radial profile
- Compute 2D fourier transform
- Compute radial profile of Mod of fourier transform -> Power spectrum



## 2-point function

In [73]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile


def f_get_power_spectrum(image,GLOBAL_MEAN=0.9998563):
    """
    Computes azimuthal average of 2D power spectrum of a np array image
    GLOBAL_MEAN is the mean pixel value of the training+validation datasets
    """
    ### Compute 2D fourier transform
    F1 = fftpack.fft2((image - GLOBAL_MEAN)/GLOBAL_MEAN)
    F2 = fftpack.fftshift(F1)
    ### Absolute value of F-transform
    pspec2d = np.abs(F2)**2
    
    ### Compute azimuthal average
    P_k = f_radial_profile(pspec2d)
    
    return P_k

In [85]:
GLOBAL_MEAN=0.9998563
F1=fftpack.fft2((img - GLOBAL_MEAN)/GLOBAL_MEAN)
F2 = fftpack.fftshift(F1)
pspec2d = np.abs(F2)**2
P_k = f_radial_profile(pspec2d)


In [86]:
plt.figure()
# plt.imshow(F.img)
plt.imshow(pspec2d)

/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
plt.figure()
# plt.plot(F1.real.flatten())
# plt.plot(F2.real.flatten())
plt.plot(P_k)

/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# plt.figure()
# plt.imshow(img)

(128, 128)

In [77]:
pwr=f_get_power_spectrum(a1[100])
plt.figure()
plt.plot(pwr,marker='o',linestyle='')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
_=f_radial_profile(img,center=(0,0))